In [1]:
!pip install chardet
!pip install stellargraph
!pip install rdkit-pypi

     |████████████████████████████████| 435 kB 536 kB/s 
     |████████████████████████████████| 5.8 MB 10.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.18.0 requires tensorflow-io-gcs-filesystem==0.18.0, which is not installed.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
beatrix-jupyterlab 3.1

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import pandas as pd
from rdkit import Chem
import numpy as np
import networkx as nx
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

import os
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings('ignore') # ignore Jupiter warnings

/kaggle/input/ai-drug-discovery/training-set.csv
/kaggle/input/ai-drug-discovery/sample-submission.csv
/kaggle/input/ai-drug-discovery/validation-set.csv



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [3]:
def one_hot_encoding(x, class_set):
    if x not in class_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, class_set))
    return list(map(lambda s: x == s, class_set))


def one_hot_encoding_unk(x, class_set):
    if x not in class_set:
        x = class_set[-1]
    return list(map(lambda s: x == s, class_set))


def atom_features(atom):
    return np.array(one_hot_encoding_unk(atom.GetSymbol(),
                                         ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As',
                                          'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se',
                                          'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                                          'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_hot_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_hot_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_hot_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    [atom.GetIsAromatic()])


def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append(tuple(feature / sum(feature)))
    features = pd.DataFrame(features)
    edges = [[bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()] for bond in mol.GetBonds()]
    g = nx.Graph(edges).to_directed()
    edge_index = pd.DataFrame([(e1, e2) for e1, e2 in g.edges]).rename(columns={0: "source", 1: "target"})
    return StellarGraph(features, edge_index)

def create_mol_dataset(dataset: pd.DataFrame):
    graphs = [smile_to_graph(smile) for index, smile in dataset['SMILES'].items()]
    graph_labels = [label for index, label in dataset['Label'].items()]
    return graphs, graph_labels


In [4]:
# read in the data
base_path = '/kaggle/input/ai-drug-discovery/'
train = pd.read_csv(base_path + 'training-set.csv')
validation = pd.read_csv(base_path + 'validation-set.csv')
train_graphs, train_labels = create_mol_dataset(train)
validation_graphs, validation_labels = create_mol_dataset(validation)

In [5]:
k = 35  # the number of rows for the output tensor
layer_sizes = [78, 156, 312]
epochs = 100
generator = PaddedGraphGenerator(train_graphs)
train_gen = generator.flow(
    range(len(train_graphs)),
    targets=train_labels,
    batch_size=50,
    symmetric_normalization=False,
)
dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["relu", "relu", "relu"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()
x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)
x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)
x_out = Flatten()(x_out)
x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)
predictions = Dense(units=1, activation="sigmoid")(x_out)
model = Model(inputs=x_inp, outputs=predictions)
model.compile(optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"])
history = model.fit(train_gen, epochs=epochs, verbose=1, shuffle=True)

2022-02-23 12:57:35.908497: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
14/14 [==============================] - 4s 69ms/step - loss: 0.6932 - acc: 0.5043
Epoch 2/100
14/14 [==============================] - 1s 67ms/step - loss: 0.6929 - acc: 0.5201
Epoch 3/100
14/14 [==============================] - 1s 68ms/step - loss: 0.6926 - acc: 0.5272
Epoch 4/100
14/14 [==============================] - 1s 64ms/step - loss: 0.6922 - acc: 0.5057
Epoch 5/100
14/14 [==============================] - 1s 62ms/step - loss: 0.6918 - acc: 0.5158
Epoch 6/100
14/14 [==============================] - 1s 63ms/step - loss: 0.6909 - acc: 0.5688
Epoch 7/100
14/14 [==============================] - 1s 62ms/step - loss: 0.6902 - acc: 0.5587
Epoch 8/100
14/14 [==============================] - 1s 61ms/step - loss: 0.6890 - acc: 0.5831
Epoch 9/100
14/14 [==============================] - 1s 64ms/step - loss: 0.6874 - acc: 0.5702
Epoch 10/100
14/14 [==============================] - 1s 61ms/step - loss: 0.6858 - acc: 0.6089
Epoch 11/100
14/14 [=============================

In [6]:
generator = PaddedGraphGenerator(validation_graphs)
test_gen = generator.flow(
    range(len(validation_graphs)),
    targets=validation_labels,
    batch_size=1,
    symmetric_normalization=False,
)
test_metrics = model.evaluate(test_gen)
test_predicts = model.predict(test_gen)
test_predicts = [int(p > 0.5) for p in test_predicts]
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))
acc = sum(1*[x[0] == x[1] for x in zip(validation_labels, test_predicts)]) / len(validation_labels)
sub = pd.read_csv(base_path + 'sample-submission.csv')
sub['Label'] = test_predicts
sub.to_csv('submission.csv', index=False)

150/150 [==============================] - 1s 3ms/step - loss: 0.7031 - acc: 0.6133

Test Set Metrics:
	loss: 0.7031
	acc: 0.6133
